# Loans 
This data set comes from BPI challenge 2017
https://www.win.tue.nl/bpi/doku.php?id=2017:challenge
You are invited to read the articles from the contesters, that relate their approach and their findings.

Download and extract this file
https://data.4tu.nl/articles/dataset/BPI_Challenge_2017/12696884/1

## Data preparation

XES files are usually never used by clients, mostly used by the process mining scientific community.
Therefore the potential pitfalls and problem met while loading XES file are not a real problem.
With this example, IBM Process Mining can raise issue while loading the file. In the end, the original data source is corrupted by wrong date formats in some rows. So anyway, we need to use another path to correct these errors.

### XES transformation
So let's transform the XES file into CSV. I selected the Python library PM4PY to do this.
It is also using Pandas dataframes. This is very convenient if you are dealing with XES files.


In [1]:
import pm4py

event_log = pm4py.read_xes("Datasets/BPI Challenge 2017.xes.gz")
df = pm4py.convert_to_dataframe(event_log)
df.to_csv('Datasets/BPI Challenge 2017.csv')
print("CSV created")

parsing log, completed traces ::   0%|          | 0/31509 [00:00<?, ?it/s]

CSV created


### Using the CSV file
Unfortunately, the generated CSV contains in some rows, related to the date format.
We thus need apply corrections first. Corrections can be applied using Excel, or any program that can load and edit from CSV files. 
In this exercise, we pursue with Pandas. Let's import the CSV file (we could just keep using the df from above).

In [2]:
import pandas

event_log = pandas.read_csv("Datasets/BPI Challenge 2017.csv",sep=',')
#The first col is useless let's drop it
event_log.drop(event_log.columns[[0]], axis=1, inplace = True) 
event_log.head()

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403000+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Correcting dates
IBM Process Mining raises date errors. <br>
Most dates are formatted like this: 2016-01-01 09:51:15.304000+00:00, where the last 3 digits of millisec are useless. <br>
There are also wrong dates are missing the milliseconds : 2016-01-02 14:00:20+00:00 <br>

We thus need to harmonize the dates. Best is to create a simple function that process the string. The function can then be applied to the entire column.

In [3]:

def harmonize_the_date(thedate) :
    if (len(thedate) == 32) :
        # 2016-01-01 09:51:15.304000+00:00
        # remove the last 3 digits of the milliseconds
        return (thedate[0 : 23] + thedate[-6:])
    else :
        # 2016-01-02 14:00:20+00:00
        # remove the end and add .000 ms
        return (thedate[0 : 19] + ".000" + thedate[-6:])
    
event_log['time:timestamp'] = event_log['time:timestamp'].apply(harmonize_the_date)

event_log.head()

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Loading in Process Mining
We now have a correct event log that IBM Process Mining can process.

There are 3 main event origins/process: Application (started by the client), Workflow (manual steps driven by a workflow), Offer (stages of offers from the bank).

It is interesting to consider the whole process with these 3 sources, but it is also interesting looking at each process individually. A_, W_, and O_ processes are simpler and could bring insights more easily.


### Let's create 3 distinct data sets
We generate one distinct CSV for each process type. We can use the field "EventOrigin" that contains either Application, Workflow, Offer. <br>
That can be easily done with these 2 lines of code, to duplicate for "Application" and "Offer" <br>
<code>w_logs = event_log[event_log["EventOrigin"].str.contains("Workflow")]
w_logs.to_csv("../Data Sets/Loans/w_events.csv")</code>

Below is a more generic approach useful when a field would have more values (using for statement)

In [4]:
origins = pandas.unique(event_log["EventOrigin"])
for originname in origins :
    origin_events = event_log[event_log["EventOrigin"].str.contains(originname)]
    origin_events.to_csv("Datasets/Loan_"+originname+"_events.csv")

### Create a process mining project for each data set
At this stage, you can start working with Process Mining and see the lifecycle and the variants of each event origin.
We have Workflow_events.csv, Application_events.csv, and Offer_events.csv.

We will come back later to the insights we can get from these files.
For the moment, let's continue our data transformation journey and see how we can declutter the original event log.


### Declutter the original file

1) Some events seem to be redundant. 
For example, "O_Create Offer" is systematically followed by "O_created" after a few milliseconds. Let's keep only the first one.
Similarly, A_Create Application and A_Submitted are redundant.

2) W_ events are created by a workflow with state changes
Each transition is logged, they might not be that much interesting.
In the end-to-end process, we could remove all these transitions, and just keeping when an activity is complete (or aborted).

Feel free to keep or remove states. The code below shows how to only keep 'complete' transitions, and how to create a mask to remove some events.

In [5]:
#Remove all events with transition <> complete or ate_abort.
complete = event_log[(event_log["lifecycle:transition"].str.contains("complete")
                      | event_log["lifecycle:transition"].str.contains("ate_abort"))]
#events with ate_abort are never complete, so they do not appear. 

#Using mask, remove all the events that statisfy this condition. Mask replaces these events with a row of 'NaN'
complete = complete.mask((event_log["concept:name"].str.contains("O_Created") 
                      | event_log["concept:name"].str.contains("A_Submitted")
                      | event_log["concept:name"].str.contains("W_Complete application")
                     ))
# remove empty rows (NaN) that were produced by mask
complete = complete[complete["concept:name"].notna()]

# save as CSV
complete.to_csv("Datasets/Loan_all_events.csv", index=False, na_rep='' )
complete

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,statechange,User_1,A_Concept,Application,ApplState_642383566,complete,2016-01-01 09:52:36.413+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,statechange,User_52,A_Accepted,Application,ApplState_99568828,complete,2016-01-02 11:23:04.299+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Created,User_52,O_Create Offer,Offer,Offer_148581083,complete,2016-01-02 11:29:03.994+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,20000.0,44.0,True,498.29,True,979.0,20000.0,NaN
11,statechange,User_52,O_Sent (mail and online),Offer,OfferState_2051164740,complete,2016-01-02 11:30:28.606+00:00,Existing loan takeover,New credit,Application_652823628,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_148581083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202256,statechange,User_96,O_Sent (mail and online),Offer,OfferState_1959881309,complete,2017-01-02 19:27:20.453+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_1580299144
1202260,statechange,User_96,A_Complete,Application,ApplState_1120616436,complete,2017-01-02 19:27:20.474+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202262,Deleted,User_1,W_Call after offers,Workflow,Workitem_1817549786,ate_abort,2017-01-06 06:33:02.212+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202264,statechange,User_28,A_Cancelled,Application,ApplState_1869071797,complete,2017-01-16 09:51:21.114+00:00,Home improvement,New credit,Application_1350494635,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
